In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [2]:
train = pd.read_csv("/kaggle/input/home-credit-manual-engineered-features/m_train_small.csv")

In [3]:
from sklearn.impute import SimpleImputer
target = train["TARGET"]
X_train = train.drop(columns = ["TARGET"])
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)


In [ ]:
from keras import backend as K
import tensorflow as tf
import keras
from keras import layers
from keras import models


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA


from imblearn.over_sampling import SMOTE

std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
pca = PCA(n_components = 25)
sm = SMOTE(k_neighbors = 110)
X_train, y_train = sm.fit_resample(X_train, target)


#X_train = pca.fit_transform(X_train)

#print(pca.explained_variance_ratio_.cumsum())





#pca = PCA(n_components = 50)

#X_train = pca.fit_transform(X_train)
#X_train = umap.UMAP(n_components=122).fit_transform(X_train)
X_train, X_test, y_train, y_test = train_test_split(
    X_train,
    y_train,
    test_size=0.1)
#X_train, y_train = sm.fit_resample(X_train, y_train)


"""
print("### AutoFeat with %i feateng_steps" % 1)
afreg = AutoFeatClassifier(verbose=1, feateng_steps=1, max_gb = 0.50, 
                           transformations = ("exp", "^2", '^3', "abs"))
df = afreg.fit_transform(X_train, y_train)
r2 = afreg.score(X_test, y_test)
print("## Final R^2: %.4f" % r2)
plt.figure()
plt.scatter(afreg.predict(X_test), y_test, s=2);
plt.title("%i FE steps (R^2: %.4f; %i new features)" % (1, r2, len(afreg.new_feat_cols_)))
"""

In [ ]:
counts = np.bincount(y_train)
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(y_train)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
model = keras.Sequential(
    [
        keras.layers.Dense(256, activation="relu", input_shape=(X_train.shape[-1],)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(4, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


In [ ]:
pd.DataFrame(X_train).isna().sum().value_counts()

In [ ]:
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
metrics = [tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tf.keras.metrics.AUC(name = "auc"),
    f1
]

model.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3), loss=f1_loss, metrics=[metrics]
)


#class_weight = {0: weight_for_0, 1: weight_for_1}

history = model.fit(
    X_train,
    y_train,
    batch_size=X_train.shape[0],
    epochs=200,
    validation_data=(X_test, y_test),
    #class_weight = class_weight
)

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history_dict["fp"]
val_acc = history_dict["val_fp"]
plt.plot(epochs, acc, "bo", label="Training fp")
plt.plot(epochs, val_acc, "b", label="Validation fp")
plt.title("Training and validation fp")
plt.xlabel("Epochs")
plt.ylabel("fp")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history_dict["fn"]
val_acc = history_dict["val_fn"]
plt.plot(epochs, acc, "bo", label="Training fn")
plt.plot(epochs, val_acc, "b", label="Validation fn")
plt.title("Training and validation fn")
plt.xlabel("Epochs")
plt.ylabel("fp")
plt.legend()
plt.show()

In [ ]:
test = pd.read_csv("/kaggle/input/home-credit-manual-engineered-features/m_test_small.csv")

In [ ]:
#test = test.align(train, axis = 1)

In [ ]:
test

In [ ]:

test = imputer.transform(test)

In [ ]:
#pca = PCA(n_components = 50)


test = std_scaler.transform(test)
#test = pca.fit_transform(test)

In [ ]:
test_model = model.predict(test)
submit = pd.read_csv("/kaggle/input/test-imputed-credit-score/test_imputed.csv")[['SK_ID_CURR']]
submit['TARGET'] = test_model

In [ ]:
submit.head()

In [ ]:
submit.to_csv("test_model_NN.csv", index = False)